In [1]:
# Copyright 2020 The HuggingFace Team All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# this notebook builds on the `run_mlm.py` example from huggingface transformers v4.16.2
# https://github.com/huggingface/transformers/blob/v4.16.2/examples/pytorch/language-modeling/run_mlm.py

## Imports and version check

In [43]:
import abc
from collections import defaultdict
import logging
import math
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
import numpy as np
from typing import Optional
from pathlib import Path

from beartype import beartype
import datasets as ds
import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoModel,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, WeightedRandomSampler

from typing import *

In [44]:
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.16.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

logger = logging.getLogger(__name__)
MODEL_CONFIG_CLASSES = list(MODEL_FOR_MASKED_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [45]:
tmp_dir = os.environ["SLURM_TMPDIR"]

## Code

### Arguments

In [46]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_overrides: Optional[str] = field(
        default=None,
        metadata={
            "help": "Override some existing default config settings when a model is trained from scratch. Example: "
            "n_embd=10,resid_pdrop=0.2,scale_attn_weights=false,summary_type=cls_index"
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

    def __post_init__(self):
        if self.config_overrides is not None and (self.config_name is not None or self.model_name_or_path is not None):
            raise ValueError(
                "--config_overrides can't be used in combination with --config_name or --model_name_or_path"
            )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    max_seq_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated."
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    mlm_probability: float = field(
        default=0.15, metadata={"help": "Ratio of tokens to mask for masked language modeling loss"}
    )
    line_by_line: bool = field(
        default=False,
        metadata={"help": "Whether distinct lines of text in the dataset are to be handled as distinct sequences."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                if extension not in ["csv", "json", "txt"]:
                    raise ValueError("`train_file` should be a csv, a json or a txt file.")
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                if extension not in ["csv", "json", "txt"]:
                    raise ValueError("`validation_file` should be a csv, a json or a txt file.")


### Custom Trainer

In [47]:
@dataclass
class RetrieverArguments:
    """
    Arguments pertaining to using a Retriever.
    """

    hf_model: Optional[str] = field(
        default="facebook/contriever", 
        metadata={"help": "Which model to use for the embeds used by the faiss retriever ('facebook/contriever' by default)."}
    )
    topk: Optional[int] = field(
        default=10, metadata={"help": "How many neighbours to retrieve for each sample (10 by default)."}
    )
    decay: Optional[float] = field(
        default=0.95, 
        metadata={"help": "Rate of decay for eligibility traces, i.e. at each time step the trace is multiplied by decay (0.95 by default)."}
    )
    use_retriever: bool = field(
        default=False, 
        metadata={"help": "Use the retriever."}
    )

In [48]:
class BaseRetriever(abc.ABC):
    @abc.abstractmethod
    def retrieve(self, query_ids, query_index):
        pass


class BoostingMLMRetriever(BaseRetriever):
    """
    This is a retriever implementation which operates on a huggingface datasets.Dataset object.
    
    Using the huggingface transformers model specified in `self.config.hf_model`, 
    this retriever tokenizes and embeds the text in the `Dataset` object passed to the `setup` method.
    The embedding approach is borrowed from https://huggingface.co/facebook/contriever 
    based on preliminary results which showed poor performance with e.g. bert.
    Using the built-in functionality of the `Dataset` object, a faiss index is made with these embeds.
    
    When retrieving examples, the index of the query in the training dataset is used (`query_index`).
    Thus, it is important that the dataset passed to `setup` include an explicit "index" col.
    This index col is shared with the `train_dataset` in the `Trainer` and ensures that 
    even if the `train_dataset` is shuffled, our `query_index` is consistent between Trainer/Retriever.
    
    Retrieved results are filtered to remove the query (while maintaining the specified `topk`).  
    
    """

    def __init__(self, config: RetrieverArguments):
        self.config = config

    @beartype
    def setup(self, dataset: ds.Dataset, device: torch.device):
        tokenizer = AutoTokenizer.from_pretrained(self.config.hf_model)
        model = AutoModel.from_pretrained(self.config.hf_model)
        model.to(device)

        def batch_embed(examples):
            indices = examples.pop("index")
            sentences = examples.pop("text")
            inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to(device)
            with torch.inference_mode():
                mask = inputs["attention_mask"]
                embeds = model(**inputs)[0].masked_fill(~mask[..., None].bool(), 0.)
                embeds = embeds.sum(dim=1) / mask.sum(dim=1)[..., None]
                embeds = embeds.cpu().numpy()
                    
            return dict(embeds=embeds, index=indices)

        dataset = dataset.map(
            batch_embed,
            batch_size=training_args.eval_batch_size,
            batched=True,
            num_proc=1, 
        )
        self.dataset = dataset.sort("index")
        self.dataset.add_faiss_index("embeds")
        
        del model
        del tokenizer
        
    @beartype
    def retrieve(self, query_ids: Any, query_index: int):
        qembed = np.array(retriever.dataset[query_index]["embeds"], dtype=np.float32)
        out = self.dataset.get_nearest_examples("embeds", qembed, k=self.config.topk+1)
        
        filtered = [i for i,idx in enumerate(out[1]["index"]) if idx != query_index]
        scores = [out[0][i] for i in filtered][:self.config.topk]
        indices = [out[1]["index"][i] for i in filtered][:self.config.topk]
        
        # NOTE: this return signature is different than the original
        return indices, scores

In [49]:
class BoostingMLMIterator(IterableDataset):
    """
    Samples next train examples based on previous train examples' losses and retrieved neighbours.
    
    The approach implemented in this class is a crude coercion of eligibility traces from RL.
    See: http://www.incompleteideas.net/book/ebook/node72.html for explanation of eligibility traces.
    
    The principal goal is to sample difficult examples more than easy examples.
    To achieve this, we keep track of an "eligibility trace" for each example, which is meant to 
    estimate its difficulty. Using `torch.utils.data.WeightedRandomSampler` we can then use these 
    eligibility traces as weights to achieve our above-stated goal.
    
    Here are some desiderata that inform this implementation:
    1. Examples with unknown difficulty should be reliably sampled (exploration);
    2. Examples known to be easy should be sampled less than unknown difficulty examples;
    3. Examples known to be difficult should correspondingly be sampled more (exploitation);
    4. As training progress, the difficulty of an example can drastically change. Therefore 
        the longer an example (or similar) goes unseen, the more its sampling probability 
        should approach that of an unknown difficulty example.
    5. Conversely, if an example (or similar) is repeatedly seen and associated with low/high 
        difficulty, the probability of sampling should decrease/increase accordingly to capture 
        the increased confidence from seeing it multiple times.
      
    Each time a difficult example is observed, we want to increase the trace of similar examples.
    Conversely, for easy examples, we want to decrease the trace of similar examples.
    Unknown or long-unseen examples should have a trace between these two.
    
    We first set the range for traces to (0,100]. 
    This prevents runaway increases in traces (from e.g. repeated sampling of difficult examples).
    It also allows us to define the midpoint (50) as a target trace for unseen/unknown examples.
    We maintain this target by incrementing every trace by `midpoint * (1-decay)` at each step, 
    allowing us to match the decay (i.e. traces converge to the midpoint from both directions).
    This satisfies desiderata 1 and 4.
    
    To satisfy desiderata 2,3 and 5, we calculate a "difficulty score" for each example returned by
    the retriever, and increment the example's trace by this score. The score of these examples is 
    negative/positive if the corresponding query example was relatively easy/hard, respectively. 
    This score is also scaled by how similar a retrieved example is to its query example. 
    The resulting scores are then rescaled to have a stdev of 25, or 1/4 of the range.
    This is arbitrary, but seems to work well in practice for obtaining well-distributed traces.
    
    """
    @beartype
    def __init__(
            self,
            *,
            dataset: ds.Dataset,
            retriever_client: BoostingMLMRetriever,
            decay: float = 0.8,
            batch_size: int = None,
    ):
        super().__init__()
        self.dataset = dataset
        self.retriever_client = retriever_client
        self.dataset_iter = None
        self.decay = decay
        self.batch_size = batch_size
        
        # eligibility trace params
        self.floor = 1e-5
        self.ceil = 100
        self.mid = self.floor + ((self.ceil - self.floor) / 2)
        self.exp_rescale = 10
        self.std = (self.ceil - self.floor) / 4
        self.eligibility = torch.ones(len(self.dataset)) * self.mid
        self.default_update = self.mid * (1 - self.decay)
        
        self.reset_sampler()

    def reset_sampler(self):
        """
        Updates the weight of the sampler using eligibility traces and samples new batch.
        Using the eligibility traces leads to very similar behavior with uniform sampling.
        To make sampling more greedy, we can exponentiate these (using `self.exp_rescale` 
        to ensure a reasonable range)
        
        """
        num_samples = self.batch_size or len(self.dataset)
        weights = torch.exp(self.eligibility / self.exp_rescale)
        self.sampler = iter(list(WeightedRandomSampler(
            weights=weights, num_samples=num_samples, replacement=False
        )))

    def push_score(self, indices: list[int], losses: torch.Tensor):
        """Use batch examples and losses to retrieve and score similar examples.
        
        NOTE: changed the signature to match call in `BoostingMLMTrainer.training_step`
        
        :param indices: train_dataset example indices from current batch
        :param losses:  average NLL losses aligned with `indices`
        """
        # Rescale losses to mean=0, std=1 so easy examples have negative scores
        # and positive examples have positive scores, in a way that is invariant 
        # to changes in the scale of batch losses throughout training.
        example_scores = (losses - losses.mean()) / losses.std()
        
        # Get loss-weighted scores for retrieved neighbours of each example.
        # These neighbours may overlap. If a neighbour occurs for multiple 
        # examples, we assume it is more relevant and sum the scores instead 
        # of averaging them. Because the normalized losses are centered on 0, 
        # a neighbour of many easy examples will have a highly negative score; 
        # a neighbour of many hard examples will have a highly positive score; 
        # and a neighbour of diverse examples will have a neutral score.
        retriever_scores = defaultdict(float)
        for idx, score in zip(indices, example_scores):
            sidxs, ssims = self.retriever_client.retrieve(None, idx)
            for sidx, ssim in zip(sidxs, ssims):
                if ssim < 0: # ignore dissimilar retrievals
                    continue
                retriever_scores[sidx] += ssim * score  
                
        indices = list(retriever_scores.keys())
        scores = torch.Tensor([retriever_scores[k] for k in indices])
        
        # Normalize scores to specified stdev and compute eligibility trace updates
        scores = scores / scores.std() * self.std
        updates = torch.ones_like(self.eligibility) * self.default_update
        updates[indices] += scores  - self.default_update
        
        # Decay and increment traces by update. Clamp to prevent runaway updates.
        self.eligibility *= self.decay
        self.eligibility += updates
        self.eligibility = torch.clamp(self.eligibility, min=self.floor, max=self.ceil)

        # Reload datasampler using new eligibility traces
        # TODO: check how slow this is, it might add a lot of overhead
        self.reset_sampler()

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        # Respect original dataset length
        if self.count == self.__len__():
            raise StopIteration
        else:
            self.count += 1
            
        # Get idx from sampler, resetting if necessary
        try:
            idx = next(self.sampler)
        except StopIteration:
            self.reset_sampler()
            idx = next(self.sampler)

        return self.dataset[idx]

In [50]:
class BoostingMLMTrainer(Trainer):
    """
    Subclass of huggingface transformers Trainer implementing MLM retriever specific functionality.
    
    The key addition is obtaining NLL losses at the example level (i.e. avg'ed over tokens in a sequence)
    aligned with the examples' indices in the train dataset, and providing these to the `push_score` 
    method of `self.train_dataset` to update its internal state and its sampling behavior.
    In particular, `BoostingMLMIterator` uses these NLL losses to weigh retriever scores 
    proportional to the difficulty of a query.
    
    These are some key assumptions that are not explicitly checked for:
    - assumes "index" is provided in the dataset for use by the retriever, otherwise breaks;
    - assumes `self.train_dataset` is an instance of `BoostingMLMIterator` or implements `push_score`;
    - assumes the number of dataset workers is 1 (`BoostingMLMIterator` doesn't support multiple workers);
    
    """
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)
        
        # NOTE: pop indices used by retriever
        indices = inputs.pop("index")
        
        # NOTE: added return_outputs obtain logits
        with self.autocast_smart_context_manager():
            loss, outputs = self.compute_loss(model, inputs, return_outputs=True)

        if self.args.n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training

        if self.args.gradient_accumulation_steps > 1 and not self.deepspeed:
            # deepspeed handles loss scaling by gradient_accumulation_steps in its `backward`
            loss = loss / self.args.gradient_accumulation_steps

        if self.do_grad_scaling:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with torch.cuda.amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            # loss gets scaled under gradient_accumulation_steps in deepspeed
            loss = self.deepspeed.backward(loss)
        else:
            loss.backward()

        # NOTE: obtaining example-level losses to update retriever        
        with torch.no_grad():
            logits = outputs["logits"].detach().transpose(1,2)  # (bsz,vsz,seq)
            labels = inputs["labels"]
            nll_counts = labels.not_equal(-100).sum(dim=1)      # (bsz)
            nll = F.nll_loss(logits, labels, reduction='none')  # (bsz,seq)
            nll = nll.sum(dim=1) / nll_counts                   # (bsz)
            self.update_iterator(indices, nll)
            
        return loss.detach()
    
    def update_iterator(self, indices: torch.Tensor, losses: torch.Tensor):
        # NOTE: convert indices to list to work with faiss when iterated over
        indices = indices.detach().cpu().tolist()
        losses = losses.detach().cpu()
        self.train_dataset.push_score(indices, losses)

## Experiment

### Set args, logging, checkpoint, and seed

In [51]:
experiment_name = "05_contriever_exp"

model_args = ModelArguments(
    config_name="bert-base-uncased",
    tokenizer_name="bert-base-uncased",
)

data_args = DataTrainingArguments(
    dataset_name="wikitext",
    dataset_config_name="wikitext-2-raw-v1",
    max_seq_length=128,
    preprocessing_num_workers=4,
)

training_args = TrainingArguments(
    per_device_train_batch_size=64,
    per_device_eval_batch_size=256,
    do_train=True,
    do_eval=True,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=100,
    output_dir=Path(tmp_dir) / f"RetroBoost/{experiment_name}",
    seed=1337, # for good luck
    log_level='warning',
    logging_steps=10,
    remove_unused_columns=False, # keep "index" col for use by retriever
    
)

retriever_args = RetrieverArguments(
    use_retriever=True,
    hf_model="facebook/contriever",
    topk=10,
    decay=0.95,
)

In [52]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
ds.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
# Set the verbosity to info of the Transformers logger (on main process only):
logger.info(f"Training/evaluation parameters {training_args}")

02/13/2022 21:50:37 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False


In [53]:
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

In [54]:
set_seed(training_args.seed)

### Get the datasets

In [58]:
# Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub
#
# For CSV/JSON files, this script will use the column called 'text' or the first column. You can easily tweak this
# behavior (see below)
#
# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = ds.load_dataset(
        data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir
    )
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = ds.load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
        )
        raw_datasets["train"] = ds.load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
        )
else:
    data_files = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
        extension = data_args.train_file.split(".")[-1]
    if data_args.validation_file is not None:
        data_files["validation"] = data_args.validation_file
        extension = data_args.validation_file.split(".")[-1]
    if extension == "txt":
        extension = "text"
    raw_datasets = ds.load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)

    # If no validation data is there, validation_split_percentage will be used to divide the dataset.
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = ds.load_dataset(
            extension,
            data_files=data_files,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
        )
        raw_datasets["train"] = ds.load_dataset(
            extension,
            data_files=data_files,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
        )

# See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
# https://huggingface.co/docs/datasets/loading_datasets.html.

02/13/2022 21:51:51 - WARNING - datasets.builder - Reusing dataset wikitext (/home/mila/m/mirceara/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

### Get the model and tokenizers

In [59]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
else:
    config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")
    if model_args.config_overrides is not None:
        logger.info(f"Overriding config: {model_args.config_overrides}")
        config.update_from_string(model_args.config_overrides)
        logger.info(f"New config: {config}")

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if model_args.model_name_or_path:
    model = AutoModelForMaskedLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForMaskedLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

### Preprocess dataset for MLM

In [60]:
# Preprocessing the datasets.
# First we tokenize all the texts.
if training_args.do_train:
    column_names = raw_datasets["train"].column_names
else:
    column_names = raw_datasets["validation"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

if data_args.max_seq_length is None:
    max_seq_length = tokenizer.model_max_length
    if max_seq_length > 1024:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --max_seq_length xxx."
        )
        max_seq_length = 1024
else:
    if data_args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

if data_args.line_by_line:
    # When using line_by_line, we just tokenize each nonempty line.
    padding = "max_length" if data_args.pad_to_max_length else False

    def tokenize_function(examples):
        # Remove empty lines
        examples[text_column_name] = [
            line for line in examples[text_column_name] if len(line) > 0 and not line.isspace()
        ]
        return tokenizer(
            examples[text_column_name],
            padding=padding,
            truncation=True,
            max_length=max_seq_length,
            # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
            # receives the `special_tokens_mask`.
            return_special_tokens_mask=True,
        )

    with training_args.main_process_first(desc="dataset map tokenization"):
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=[text_column_name],
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on dataset line_by_line",
        )
else:
    # Otherwise, we tokenize every text, then concatenate them together before splitting them in smaller parts.
    # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
    # efficient when it receives the `special_tokens_mask`.
    def tokenize_function(examples):
        return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

    with training_args.main_process_first(desc="dataset map tokenization"):
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on every text in dataset",
        )

    # Main data processing function that will concatenate all texts from our dataset and generate chunks of
    # max_seq_length.
    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
        if total_length >= max_seq_length:
            total_length = (total_length // max_seq_length) * max_seq_length
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
            for k, t in concatenated_examples.items()
        }
        return result

    # Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
    # remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
    # might be slower to preprocess.
    #
    # To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
    # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

    with training_args.main_process_first(desc="grouping texts together"):
        tokenized_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
            desc=f"Grouping texts in chunks of {max_seq_length}",
        )

if training_args.do_train:
    if "train" not in tokenized_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = tokenized_datasets["train"]
    if data_args.max_train_samples is not None:
        train_dataset = train_dataset.select(range(data_args.max_train_samples))

if training_args.do_eval:
    if "validation" not in tokenized_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = tokenized_datasets["validation"]
    if data_args.max_eval_samples is not None:
        eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))

# NOTE: add index for use by trainer to call retriever
train_dataset = train_dataset.add_column("index", list(range(len(train_dataset))))

# NOTE: add back original text for use by retriever (with different tokenizer)
def batch_decode_for_retriever(examples):
    sentences = tokenizer.batch_decode(examples['input_ids'], skip_special_tokens=True)
    results = dict(
        text=sentences,
        index=examples["index"]
    )
    return results

with training_args.main_process_first(desc="Adding texts back for retriever"):
    remove_cols = [c for c in train_dataset.column_names if c != "index"]
    retriever_dataset = train_dataset.map(
        batch_decode_for_retriever,
        batched=True,
        num_proc=data_args.preprocessing_num_workers,
        remove_columns=remove_cols,
        load_from_cache_file=not data_args.overwrite_cache,
        desc=f"Adding texts back for retriever",
    )
    
# Data collator
# This one will take care of randomly masking the tokens.
pad_to_multiple_of_8 = data_args.line_by_line and training_args.fp16 and not data_args.pad_to_max_length
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=data_args.mlm_probability,
    pad_to_multiple_of=8 if pad_to_multiple_of_8 else None,
)


02/13/2022 21:51:54 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/mila/m/mirceara/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-7d2c256f2eaac353.arrow
02/13/2022 21:51:54 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/mila/m/mirceara/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f97474a0f7ddcb44.arrow
02/13/2022 21:51:54 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/mila/m/mirceara/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d8dbee640d83fa33.arrow
02/13/2022 21:51:54 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/mila/m/mirceara/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf

### Initialize Trainer

In [61]:
# TODO: cache retriever embeds so they don't have to be recomputed each time
if retriever_args.use_retriever:
    retriever = BoostingMLMRetriever(retriever_args)
    retriever.setup(retriever_dataset, torch.device("cuda:0"))
    train_dataset = BoostingMLMIterator(
        dataset=train_dataset, 
        retriever_client=retriever, 
        decay=retriever_args.decay,
        batch_size=training_args.train_batch_size)
    trainer_class = BoostingMLMTrainer
else:
    trainer_class = Trainer

  0%|          | 0/73 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [62]:
# Initialize our Trainer
trainer = trainer_class(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

### Training

In [63]:
# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()  # Saves the tokenizer too for easy upload
    metrics = train_result.metrics

    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

/home/mila/m/mirceara/.conda/envs/rb/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,7.515300,7.362861
200,7.168000,7.107363
300,7.071000,7.000046
400,7.009900,6.916716
500,6.969400,6.872277


***** train metrics *****
  epoch                    =        2.0
  total_flos               =  2273445GF
  train_loss               =     7.2714
  train_runtime            = 0:16:31.23
  train_samples            =      18549
  train_samples_per_second =     37.426
  train_steps_per_second   =      0.585


In [64]:
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    metrics = trainer.evaluate()

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
    try:
        perplexity = math.exp(metrics["eval_loss"])
    except OverflowError:
        perplexity = float("inf")
    metrics["perplexity"] = perplexity

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        2.0
  eval_loss               =     6.8699
  eval_runtime            = 0:00:05.42
  eval_samples            =       1921
  eval_samples_per_second =    354.235
  eval_steps_per_second   =      1.475
  perplexity              =   962.8759
